# Backup: Final Version for Tracking (May 10, 9:30pm)

In [ ]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import time

def sum_abs_diff(image1, image2):
    image1 = image1.astype('int32')
    image2 = image2.astype('int32')
    
    sad = 0
    
    if image1.shape == image2.shape:
        diff = image1 - image2
        sad = np.sum(np.absolute(diff))
    else:
        sad = -1
    
    return sad

def scan_line(span, template, search_col_min, search_col_max):
    min_place = -1
    min_value = float('inf')
    if search_col_min >= search_col_max:
        return (False, 0)
    for i in range(search_col_min, search_col_max):
        diff = sum_abs_diff(span[:, i:i + span.shape[0]], template)
        if diff < min_value:
            min_value = diff
            min_place = i
    return (min_place, min_value)

def Depth(leftGrayImg, rightGrayImg, positionOnRight, halfWindow):
    baseline = 120
    focalLength = 700
    
    template = rightGrayImg[positionOnRight[0]-halfWindow:positionOnRight[0]+halfWindow,\
                           positionOnRight[1]:positionOnRight[1]+halfWindow*2]
        
    span = leftGrayImg[positionOnRight[0]-halfWindow:positionOnRight[0]+halfWindow, :]
    
    min_place, min_value = scan_line(span, template, positionOnRight[1]+halfWindow,
                                     min(positionOnRight[1]+250, leftGrayImg.shape[1] - 2*halfWindow))
    if min_place < 0:
        return 0;
        
    disparity = min_place - positionOnRight[1]
#     print(f"Disparity: {disparity} pixels")
    
    depth = focalLength*baseline/disparity
#     print(f"Depth of pixel [{positionOnRight[0]},{positionOnRight[1]}] in mm: {depth}")
    return depth

def BackgroundRemoval(rightImages, occlusion):
    out = cv2.VideoWriter('WithoutOcclusionFinal.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1280, 720))
    
    # Initialize Kalman filter
    stateSize = 7
    measSize = 5
    contrSize = 0
    kalman = cv2.KalmanFilter(stateSize, measSize, contrSize, cv2.CV_32F)
    state = np.zeros((stateSize, 1), np.float32) # x,y,v_x,v_y,w,h,d,v_d
    meas = np.zeros((measSize, 1), np.float32) # x,y,w,h,d
    kalman.measurementMatrix = np.array([[1,0,0,0,0,0,0],
                                         [0,1,0,0,0,0,0],
                                         [0,0,0,0,1,0,0],
                                         [0,0,0,0,0,1,0],
                                         [0,0,0,0,0,0,1]], np.float32)
    kalman.transitionMatrix = np.array([[1,0,1,0,0,0,0],
                                        [0,1,0,1,0,0,0],
                                        [0,0,1,0,0,0,0],
                                        [0,0,0,1,0,0,0],
                                        [0,0,0,0,1,0,0],
                                        [0,0,0,0,0,1,0],
                                        [0,0,0,0,0,0,1]], np.float32)
    kalman.processNoiseCov = np.array([[0.01,0,0,0,0,0,0],
                                       [0,0.01,0,0,0,0,0],
                                       [0,0,5,0,0,0,0],
                                       [0,0,0,5,0,0,0],
                                       [0,0,0,0,0.01,0,0],
                                       [0,0,0,0,0,0.01,0],
                                       [0,0,0,0,0,0,0.1]], np.float32)
    kalman.measurementNoiseCov = np.eye(measSize, dtype=np.float32) * 0.1
    
    # Filtering
    fgbg = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)  
    # Define parameters to check it is new object?
    thresholdCenter = 500
    prevCenter = (0, 0)
    prevArea = 0
    thresholdArea = 2500

    # Needed for the area calculation to work
    dismissFrames = 20
    counter = 0
    
    ratio = 0
    ratioStart = 0
    found = False
    oCounter = 0
    objectFound = False
    afterOcclusion = False
    prevWidth = 0
    
    for rightName in rightImages:
        counter += 1
        leftName = rightName.replace('right','left').replace('Right','Left')
        
        # Read left frame to get object detection running
        originalFrame = cv2.imread(rightName)
        frame = originalFrame.copy()
        rightGray = cv2.cvtColor(originalFrame, cv2.COLOR_BGR2GRAY)
                
        if found:
            # Make a prediction
            state = kalman.predict()
            predRect = (state[0], state[1] - state[5]/2, state[4], state[5])
            center = (state[0], state[1])
            cv2.putText(originalFrame, f"Predictions: x={state[0]}, y={state[1]}, depth={state[6]}",
                       (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
            cv2.circle(originalFrame, center, 5, (255,0,0), -1)
            cv2.rectangle(originalFrame, predRect, (255,0,0), 2)
                
        # Get the mask of the object
        mask = fgbg.apply(frame)
        
        # Erode it and dilate it to avoid small points
        kernel = np.ones((5,5), np.uint8) 
        mask = cv2.erode(mask, kernel, iterations=2) 
        mask = cv2.dilate(mask, kernel, iterations=2) 

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(originalFrame, originalFrame, mask=mask)
        
        # Make a crop to focus in the moving thing
        cropTop = 200
        cropBottom = 675
        cropLeft = 0#200
        cropRight = 1280#1150#1280
        cropped = mask[cropTop:cropBottom, cropLeft:cropRight]
        
        if counter > dismissFrames:
            cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

            if len(cnts) > 0:

                c = max(cnts, key=cv2.contourArea)
                area = cv2.contourArea(c)
                
                x,y,width,height = cv2.boundingRect(c)
                cv2.rectangle(originalFrame, (x+cropLeft,y+cropTop,width,height), (0, 255, 0))
                
                M = cv2.moments(c)
                if M["m00"] == 0.0:
                    continue
                center = (int(M["m10"] / M["m00"]) + cropLeft, int(M["m01"] / M["m00"]) + cropTop)            

                if area > thresholdArea:
                    currentRatio = float(width)/float(height)
                    # Check for new object
                    if center[0] > prevCenter[0] + thresholdCenter:
                        print("New object!")
                        afterOcclusion = False
                        gettingBehindOcclusion = False
                        found = False
                        oCounter = 30
                    # Check whether object reappeared on the left side of the occlusion
                    elif center[0] < prevCenter[0] - 300:
                        afterOcclusion = True
                        
                    left = cv2.imread(leftName)
                    leftGray = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)

                    meas[0] = x + cropLeft
                    meas[1] = y + cropTop + height/2.0
                    meas[2] = float(width)
                    meas[3] = float(height)
                    meas[4] = Depth(leftGray, rightGray, (int(meas[1]), int(meas[0])), 10)
                    
                    writeMeas = True

                    # After occlusion
                    if afterOcclusion:
                        kalman.correct(meas)
                    
                    # Before occlusion
                    elif oCounter > 0:
                        writeMeas = True
                        ratio = currentRatio
                        oCounter -= 1
                        if not found:
                            kalman.errorCovPre = np.eye(stateSize, dtype=np.float32)
                            state[0] = meas[0]
                            state[1] = meas[1]
                            state[2] = 0
                            state[3] = 0
                            state[4] = meas[2]
                            state[5] = meas[3]
                            state[6] = meas[4]
                            kalman.statePost = state
                            found = True
                        else:
                            kalman.correct(meas)
                    elif occlusion:
                        # Checking if occlusion started
                        if abs(currentRatio-ratio) < 0.05 and not gettingBehindOcclusion:
                            cv2.circle(originalFrame, (meas[0], meas[1]), 2, (0, 255, 0), -1)
                            kalman.correct(meas)

                        # Otherwise, the object is getting behind the occlusion
                        else:
                            gettingBehindOcclusion = True
                            writeMeas = False
                    else:
                        cv2.circle(originalFrame, (meas[0], meas[1]), 2, (0, 255, 0), -1)
                        kalman.correct(meas)

                    prevCenter = center
                    if writeMeas:
                        cv2.putText(originalFrame, f"Measurement: x={meas[0]}, y={meas[1]}, d={meas[4]}",
                                   (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0))
                else:
                    pass
            else: 
                pass

            cv2.imshow('res', originalFrame)
            out.write(originalFrame)

        #press 'q' to kill
        if cv2.waitKey(15) & 0xFF == ord('q'):
            break
    
    out.release()
    
    cv2.destroyAllWindows()
    return 

occlusion = not True
if occlusion:
    basename = 'undistortedImagesWithOcclusion/right/*.png'
else:
    basename = 'undistortedImagesWithoutOcclusion/right/*.png'

rightImages = glob.glob(basename)
assert rightImages

BackgroundRemoval(rightImages, occlusion)

# Older versions

In [ ]:
import cv2
import numpy as np
import imutils




def BackgroundRemoval(cap, fgbg_params):
    # Define kalman
    kalman = cv2.KalmanFilter(4,2)
    kalman.measurementMatrix = np.array([[1,0,0,0],
                                         [0,1,0,0]],np.float32)
    kalman.transitionMatrix = np.array([[1,0,1,0],
                                        [0,1,0,1],
                                        [0,0,1,0],
                                        [0,0,0,1]],np.float32)
    kalman.processNoiseCov = np.array([[1,0,0,0],
                                       [0,1,0,0],
                                       [0,0,1,0],
                                       [0,0,0,1]],np.float32) * 0.03
    measurement = np.array((2,1), np.float32)
    prediction = np.zeros((2,1), np.float32)
    
    # Background removal
    fgbg = cv2.createBackgroundSubtractorKNN(**fgbg_params)
    
    # Define parameters to check it is new object?
    thresholdCenter = 10
    prevCenter = (0, 0)
    prevRadius = 0
    thresholdRadius = 5
    
    while(1):
        # Take each frame
        ret, originalFrame = cap.read()
        
        # Make a prediction
        prediction = kalman.predict()
        
        if ret == False:
            break
        
        # Get the mask of the object
        mask = fgbg.apply(originalFrame)
        
        # Erode it and dilate it to avoid small points
        kernel = np.ones((5,5), np.uint8) 
        mask = cv2.erode(mask, kernel, iterations=1) 
        mask = cv2.dilate(mask, kernel, iterations=1) 

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(originalFrame, originalFrame, mask= mask)
        
        # Make a crop to focus in the moving thing
        cropTop = 200
        cropBottom = 600
        cropLeft = 300
        cropRight = 1150
        cropped = mask[cropTop:cropBottom, cropLeft:cropRight]
        
        # Find contours
        cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Only proceed if at least one contour was found
        if len(cnts) > 0:
            # find the largest contour in the mask, then use
            # it to compute the minimum enclosing circle and
            # centroid
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            if M["m00"] == 0.0:
                continue
            center = (int(M["m10"] / M["m00"])+cropLeft, int(M["m01"] / M["m00"])+cropTop)
            
            # only proceed if the radius meets a minimum size
            if radius > 20 and radius < 200 and radius + thresholdRadius >= prevRadius:
                # draw the circle and centroid on the frame,
                # then update the list of tracked points
                cv2.circle(originalFrame, (int(x)+cropLeft, int(y)+cropTop), int(radius),
                    (0, 255, 255), 2)
                cv2.circle(originalFrame, center, 5, (0, 0, 255), -1)
                    
                # Correct kalman 
                kalman.correct(np.array([np.float32(center[0]), np.float32(center[1])], np.float32))
                
                prevRadius = radius
                text = "Object found at:"
                colour = (0, 255, 0)
                cv2.putText(originalFrame,"({}, {})".format(center[0], center[1]), (0, 100), cv2.FONT_ITALIC, 1, colour)

#                 h, w, channels = originalFrame.shape
#                 h = radius
#                 w = radius

                if (center[0] > prevCenter[0] + thresholdCenter) and (center[1] < prevCenter[1] - thresholdCenter):
                    # New object
                    print("New object!")
                    objectFound = False
                    prevRadius = 0
                prevCenter = center
            else:
                # Object not found
                text = "Object not found"
                objectFound = False
                colour = (0, 0, 255)
                
        
        #
        else: 
            # Object not found
            text = "Object not found"
            objectFound = False
            colour = (0, 0, 255)
            prevRadius = 0
            
        prediction = kalman.predict()
#         originalFrame = cv2.rectangle(originalFrame, (prediction[0]-(0.5*w), prediction[1]-(0.5*h)), 
#                                 (prediction[0]+(0.5*w), prediction[1]+(0.5*h)), (0,255,0),2)
        cv2.circle(originalFrame, (prediction[0]+prediction[2], prediction[1]+prediction[3]), 5, (255, 0, 0), -1)
        cv2.putText(originalFrame,"({}, {}, {}, {})".format(prediction[0], prediction[1], prediction[2], prediction[3]), 
                    (0, 150), cv2.FONT_ITALIC, 1, colour)
    
            
        cv2.putText(originalFrame, text, (0, 50), cv2.FONT_ITALIC, 1, colour)


        cv2.imshow('res',originalFrame)

        #press 'q' to kill
        if cv2.waitKey(15) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return 

# params for Background Removal
fgbg_params = dict( history=1000, 
                    dist2Threshold=1000, 
                    detectShadows=False )

cap = cv2.VideoCapture('Stereo_conveyor_with_occlusions.mp4')

BackgroundRemoval(cap, fgbg_params)

In [1]:
import cv2
import sys
import numpy as np

tracker = cv2.TrackerTLD_create()

trackingStarted = False

video = cv2.VideoCapture("Stereo_conveyor_with_occlusions.mp4")

fgbg = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)

prevCenter = (0,0)

while(1):
    ret, originalFrame = video.read()
    mask = fgbg.apply(originalFrame)
    kernel = np.ones((5,5), np.uint8) 
    mask = cv2.erode(mask, kernel, iterations=1) 
    mask = cv2.dilate(mask, kernel, iterations=1) 
    frame = cv2.bitwise_and(originalFrame, originalFrame, mask= mask)
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        area = cv2.contourArea(c)
        if area > 2500:
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            if center[0] > prevCenter[0] + 500:
                #new object
                rect = cv2.boundingRect(c)
                cv2.rectangle(originalFrame, rect, (255,0,0), 2)
                trackingStarted = True
                tracker.init(originalFrame, rect)
    if  trackingStarted:
        ok, box = tracker.update(originalFrame)
        p1 = (int(box[0]), int(box[1]))
        p2 = (int(box[0])+int(box[2]), int(box[1])+int(box[3]))
        if ok:
            cv2.rectangle(originalFrame, p1, p2, (0,255,0), 2)
        else:
            cv2.putText(originalFrame, "Tracking failure detected", (0,100), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
    cv2.imshow("Tracking", originalFrame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Backup May 9, 17:30 best so far

In [2]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import time

def sum_abs_diff(image1, image2):
    image1 = image1.astype('int32')
    image2 = image2.astype('int32')
    
    sad = 0
    
    if image1.shape == image2.shape:
        diff = image1 - image2
        sad = np.sum(np.absolute(diff))
    else:
        sad = -1
    
    return sad

def scan_line(span, template, search_col_min, search_col_max):
    min_place = -1
    min_value = float('inf')
    for i in range(search_col_min, search_col_max):
        diff = sum_abs_diff(span[:, i:i + span.shape[0]], template)
        if diff < min_value:
            min_value = diff
            min_place = i
    return (min_place, min_value)

def Depth(leftGrayImg, rightGrayImg, positionOnLeft, halfWindow):
    baseline = 120
    focalLength = 700
    
    template = leftGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow,\
                           positionOnLeft[0]-halfWindow:positionOnLeft[0]+halfWindow]    
    span = rightGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow, :]
    min_place, min_value = scan_line(span, template, positionOnLeft[0]-250, positionOnLeft[0]-50-halfWindow)
    
    if min_place < 0:
        print('fail')
        
    disparity = positionOnLeft[0]-min_place
    #print(f"Disparity: {disparity} pixels")
    
    depth = focalLength*baseline/disparity
    #print(f"Depth of pixel [{positionOnLeft[0]},{positionOnLeft[1]}] in mm: {depth}")
    return depth

    


def BackgroundRemoval(rightImages, occlusion):
    out = cv2.VideoWriter('tracking.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1280, 720))
    
    # Initialize Kalman filter
    stateSize = 6
    measSize = 4
    contrSize = 0
    kalman = cv2.KalmanFilter(stateSize, measSize, contrSize, cv2.CV_32F)
    state = np.zeros((stateSize, 1), np.float32) # x,y,v_x,v_y,w,h
    meas = np.zeros((measSize, 1), np.float32) # x,y,w,h
    kalman.measurementMatrix = np.array([[1,0,0,0,0,0],
                                         [0,1,0,0,0,0],
                                         [0,0,0,0,1,0],
                                         [0,0,0,0,0,1]], np.float32)
    kalman.transitionMatrix = np.array([[1,0,1,0,0,0],
                                        [0,1,0,1,0,0],
                                        [0,0,1,0,0,0],
                                        [0,0,0,1,0,0],
                                        [0,0,0,0,1,0],
                                        [0,0,0,0,0,1]], np.float32)
    kalman.processNoiseCov = np.array([[0.01,0,0,0,0,0],
                                       [0,0.01,0,0,0,0],
                                       [0,0,5,0,0,0],
                                       [0,0,0,5,0,0],
                                       [0,0,0,0,0.01,0],
                                       [0,0,0,0,0,0.01]], np.float32)
    kalman.measurementNoiseCov = np.eye(measSize, dtype=np.float32) * 0.1
    
    # Filtering
    fgbg = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)  
    # Define parameters to check it is new object?
    thresholdCenter = 500
    prevCenter = (0, 0)
    prevArea = 0
    thresholdArea = 2500

    # Needed for the area calculation to work
    dismissFrames = 20
    counter = 0
    
    ratio = 0
    ratioStart = 0
    found = False
    oCounter = 0
    objectFound = False
    afterOcclusion = False
    prevWidth = 0
    
    for rightName in rightImages:
        counter += 1
        leftName = rightName.replace('right','left').replace('Right','Left')
        
        # Read left frame to get object detection running
        originalFrame = cv2.imread(rightName)
        frame = originalFrame.copy()
        rightGray = cv2.cvtColor(originalFrame, cv2.COLOR_BGR2GRAY)
                
        if found:
            # Make a prediction
            state = kalman.predict()
            predRect = (state[0], state[1] - state[5]/2, state[4], state[5])
            center = (state[0], state[1])
            cv2.circle(originalFrame, center, 5, (255,0,0), -1)
            cv2.rectangle(originalFrame, predRect, (255,0,0), 2)
                
        # Get the mask of the object
        mask = fgbg.apply(frame)
        
        # Erode it and dilate it to avoid small points
        kernel = np.ones((5,5), np.uint8) 
        mask = cv2.erode(mask, kernel, iterations=2) 
        mask = cv2.dilate(mask, kernel, iterations=2) 

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(originalFrame, originalFrame, mask=mask)
        
        # Make a crop to focus in the moving thing
        cropTop = 200
        cropBottom = 675
        cropLeft = 200
        cropRight = 1280
        cropped = mask[cropTop:cropBottom, cropLeft:cropRight]
        
        if counter > dismissFrames:
            cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

            if len(cnts) > 0:

                c = max(cnts, key=cv2.contourArea)
                area = cv2.contourArea(c)
                
                x,y,width,height = cv2.boundingRect(c)
                cv2.rectangle(originalFrame, (x+cropLeft,y+cropTop,width,height), (0, 255, 0))
                
                M = cv2.moments(c)
                if M["m00"] == 0.0:
                    #prediction = kalman.predict()
                    continue
                center = (int(M["m10"] / M["m00"]) + cropLeft, int(M["m01"] / M["m00"]) + cropTop)            

                if area > thresholdArea:
                    currentRatio = float(width)/float(height)
                    # Check for new object
                    if center[0] > prevCenter[0] + thresholdCenter:
                        print("New object!")
                        afterOcclusion = False
                        gettingBehindOcclusion = False
                        found = False
                        oCounter = 30
                    # Check whether object reappeared on the left side of the occlusion
                    elif center[0] < prevCenter[0] - 300:
                        afterOcclusion = True
                    
                    meas[0] = x + cropLeft
                    meas[1] = y + cropTop + height/2.0
                    meas[2] = float(width)
                    meas[3] = float(height)
                    
                    # After occlusion
                    if afterOcclusion:
                        kalman.correct(meas)
                    
                    # Before occlusion
                    if oCounter > 0:
                        ratio = currentRatio
                        oCounter -= 1
                        if not found:
                            kalman.errorCovPre = np.eye(stateSize, dtype=np.float32)
                            state[0] = meas[0]
                            state[1] = meas[1]
                            state[2] = 0
                            state[3] = 0
                            state[4] = meas[2]
                            state[5] = meas[3]
                            kalman.statePost = state
                            found = True
                        else:
                            kalman.correct(meas)
                    elif occlusion:
                        # Checking if occlusion started
                        if currentRatio > ratio - 0.05 and currentRatio < ratio + 0.05 and not gettingBehindOcclusion:
                            cv2.circle(originalFrame, center, 2, (0, 255, 0), -1)
                            kalman.correct(meas)
                            #ratio = currentRatio

                        # Otherwise, the object is getting behind the occlusion
                        else:
                            gettingBehindOcclusion = True
                    else:
                        cv2.circle(originalFrame, (meas[0], meas[1]), 2, (0, 255, 0), -1)
                        kalman.correct(meas)

                    prevCenter = center
                    prevArea = area
                else:
                    pass
            else: 
                pass

            cv2.imshow('res', originalFrame)
            out.write(originalFrame)

        #press 'q' to kill
        if cv2.waitKey(15) & 0xFF == ord('q'):
            break
    
    out.release()
    
    cv2.destroyAllWindows()
    return 

occlusion = True
if occlusion:
    basename = 'undistortedImagesWithOcclusion/right/*.png'
else:
    basename = 'undistortedImagesWithoutOcclusion/right/*.png'

rightImages = glob.glob(basename)
assert rightImages

BackgroundRemoval(rightImages, occlusion)

New object!
New object!
New object!
New object!
New object!
New object!
